In [3]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

character(0)

# 1. Training Data Cleaning

Loading data

In [6]:
data <- read.csv("../input/covid19-dataset/COVID19_training_validation_set_original.csv", header=TRUE, sep=",")
summary(data)
dim(data)

       X          case_in_country  reporting.date          country   
 Min.   :   1.0   Min.   : 1.000   Min.   : 3.000   Japan      :209  
 1st Qu.: 390.5   1st Qu.: 1.000   1st Qu.: 7.250   China      :154  
 Median : 908.0   Median : 3.000   Median :11.000   Germany    :138  
 Mean   :1507.3   Mean   : 3.329   Mean   : 9.784   USA        :113  
 3rd Qu.:2945.5   3rd Qu.: 4.000   3rd Qu.:12.000   Spain      : 95  
 Max.   :3396.0   Max.   :10.000   Max.   :14.000   South Korea: 89  
                  NA's   :154      NA's   :1        (Other)    :461  
    gender         age        symptom_onset    If_onset_approximated
 female:411   Min.   :1.000   Min.   : 0.000   Min.   :0.0000       
 male  :570   1st Qu.:4.000   1st Qu.: 5.000   1st Qu.:0.0000       
 NA's  :278   Median :6.000   Median : 7.000   Median :0.0000       
              Mean   :5.393   Mean   : 6.981   Mean   :0.0343       
              3rd Qu.:7.000   3rd Qu.:10.000   3rd Qu.:0.0000       
              Max.   :8.00

[1] 1259   17

3 classes: 'dead', 'onTreatment', 'recovered'. Highly unbalanced problem.

Cleaning the data:
1. Change labels from 'male', 'female' to '0' or '1';
2. Group data by country
3. Replace 'NA' by '-1'

In [10]:
data$X <- NULL # removing this column

# Altering "gender" column: "male" <- 0, "female" <- 1
data$gender <- as.character(data$gender)
data[data$gender=="male" & !is.na(data$gender), "gender"] <- 0
data[data$gender=="female" & !is.na(data$gender), "gender"] <- 1
data$gender <- as.integer(data$gender)

# Grouping data by country
colunas <- colnames(data)[colnames(data)!="country" & colnames(data)!="label"]
paises <- unique(data$country)
medianasPorPais <- aggregate(data[,colunas], list(data$country), function(x) round(median(x, na.rm=TRUE)))

for (coluna in colunas){
  medianasPorPais[is.na(medianasPorPais[,coluna]),coluna]<- -1
  for (pais in paises){
    data[is.na(data[coluna])==TRUE & data$country==pais, coluna] <- medianasPorPais[medianasPorPais[,1]==pais, coluna]
  }
}                         

Missing values (NA):

In [11]:
sum(data == -1)

[1] 3154

To treat missing data, it was chosen to infer their values from other columns.

If "visiting.Wuhan" == 1, then "traveler" must be = 1

In [12]:
data[data$visiting.Wuhan == 1 & data$traveler == -1, "traveler"] <- 1
sum(data == -1)

[1] 3078

If "international_taveler" == 1 or "domestic_traveler" == 1 -> "traveler" == 1

In [13]:
data[(data$international_traveler == 1 || data$traveler == 1) & data$traveler == -1, "traveler"] <- 1
sum(data == -1)

[1] 2881

If "country" != China and the person either visited or is from Wuhen -> "international_traveler" = 1

In [14]:
data[data$country != "China" & (data$visiting.Wuhan == 1 || data$from.Wuhan == 1) & data$international_traveler == -1,"international_traveler"] <- 1
sum(data == -1)

[1] 1889

If "country" == China and the person visited Wuhan  -> "domestic_traveler" = 1

In [15]:
data[data$country == "China" & data$visiting.Wuhan == 1 & data$domestic_traveler == -1,"domestic_traveler"] <- 1
sum(data == -1)

[1] 1817

It is seen that columns "case_in_country" and "exposure_start" are related to "reporting.date":

In [18]:
table(data$reporting.date)
table(data$case_in_country)
table(data$exposure_start)


  3   4   5   6   7   8   9  10  11  12  13  14 
  2  19 160  84  50  74  60 132 228 245 187  18 


 -1   1   2   3   4   5   6   7   8   9  10 
154 291 172 277 116  80  48  30  23  14  54 


 -1   0   1   2   3   4   5   6   7   9  10  11  13 
183   4   3 138  26 409 259   2   1   8 218   7   1 

In [20]:
categorias <- aggregate(data[data$case_in_country != -1, "case_in_country"], list(data[data$case_in_country != -1, "reporting.date"]), median)
categorias

Group.1,x
<dbl>,<dbl>
3,1
4,1
5,1
6,1
7,1
8,2
9,3
10,3
11,3


Replacing NA values by the median:

In [21]:
for (repDate in unique(data$reporting.date)){
  data[data$case_in_country == -1 & data$reporting.date == repDate, "case_in_country"] <- categorias[categorias[,1] == repDate, 2]
}

In [22]:
categorias <- aggregate(data[data$exposure_start != -1, "exposure_start"], list(data[data$exposure_start != -1, "reporting.date"]), median)

for (repDate in unique(data$reporting.date)){
  data[data$exposure_start == -1 & data$reporting.date == repDate, "exposure_start"] <- categorias[categorias[,1] == repDate, 2]
}

In [23]:
sum(data == -1)

[1] 1480

It can be seen that all fields where 'age' == -1 present 'case_in_country' == 1

In [24]:
table(data[data$age == -1, "case_in_country"])
tab <- table(data[data$case_in_country == 1, "age"])
num <- names(tab[tab==max(tab)])


 1 
31 

In [25]:
data[data$age==-1 & data$case_in_country==1, "age"] <- num
data$age <- as.numeric(data$age)

sum(data == -1)

[1] 1449

Replacing all remaining 'NA' by the column median:

In [26]:
colunas <- 1:15; 
for (i in colunas[-3]){
  mediana <- median(data[data[,i] != -1, i])
  data[data[,i] == -1, i] <- mediana
}

**Train/validation split**

In [27]:
randomTrainIndexes <- sample(1:nrow(data), size=0.8*nrow(data))
dataTrain_opt <- data[randomTrainIndexes,]
dataVal_opt <- data[-randomTrainIndexes,]

# 2. Test Data Cleaning

Doing the same procedure for Test Data:

In [28]:
rm(colunas)
rm(paises)
rm(medianasPorPais)
rm(categorias)
rm(tab)
rm(num)

# Importando os dados de Teste
data_test <- read.csv("../input/covid19-dataset/COVID19_test_set_original.csv", header=TRUE, sep=",")
summary(data_test)
dim(data_test)

# Removendo coluna "X":
data_test$X <- NULL

# Alterando a classe da coluna "gender": "male" <- 0, "female" <- 1
data_test$gender <- as.character(data_test$gender)
data_test[data_test$gender=="male" & !is.na(data_test$gender), "gender"] <- 0
data_test[data_test$gender=="female" & !is.na(data_test$gender), "gender"] <- 1
data_test$gender <- as.integer(data_test$gender)


# Agrupando dados médios por país
colunas <- colnames(data_test)[colnames(data_test)!="country" & colnames(data_test)!="label"]
paises <- unique(data_test$country)
medianasPorPais <- aggregate(data_test[,colunas], list(data_test$country), function(x) round(median(x, na.rm=TRUE)))

for (coluna in colunas){
  medianasPorPais[is.na(medianasPorPais[,coluna]),coluna]<- -1
  for (pais in paises){
    data_test[is.na(data_test[coluna])==TRUE & data_test$country==pais, coluna] <- medianasPorPais[medianasPorPais[,1]==pais, coluna]
  }
}


# Se coluna "visiting.Wuhan" == 1 -> "traveler" deve ser = 1
data_test[data_test$visiting.Wuhan == 1 & data_test$traveler == -1, "traveler"] <- 1

# Se coluna "traveler" == 0 -> "international_traveler" e "domestic_traveler" também devem ser = 0:
data_test[data_test$traveler == 0 & data_test$international_traveler == -1, "international_traveler"] <- 0
data_test[data_test$traveler == 0 & data_test$domestic_traveler == -1, "domestic_traveler"] <- 0

# Se coluna "international_taveler" == 1 ou "domestic_traveler" == 1 -> "traveler" == 1
data_test[(data_test$international_traveler == 1 || data_test$traveler == 1) & data_test$traveler == -1, "traveler"] <- 1

# Se coluna "country" != China e a pessoa visitou Wuhan ou é de Wuhan -> "international_traveler" = 1
data_test[data_test$country != "China" & (data_test$visiting.Wuhan == 1 || data_test$from.Wuhan == 1) & data_test$international_traveler == -1,"international_traveler"] <- 1

# Se coluna "country" == China e a pessoa visitou Wuhan -> "domestic_traveler" = 1
data_test[data_test$country == "China" & data_test$visiting.Wuhan == 1 & data_test$domestic_traveler == -1,"domestic_traveler"] <- 1

categorias <- aggregate(data_test[data_test$case_in_country != -1, "case_in_country"], list(data_test[data_test$case_in_country != -1, "reporting.date"]), median)

for (repDate in unique(data_test[data_test$case_in_country != -1, "reporting.date"])){
  data_test[data_test$case_in_country == -1 & data_test$reporting.date == repDate, "case_in_country"] <- categorias[categorias[,1] == repDate, 2]
}

categorias <- aggregate(data_test[data_test$exposure_start != -1, "exposure_start"], list(data_test[data_test$exposure_start != -1, "reporting.date"]), median)

for (repDate in unique(data_test$reporting.date)){
  data_test[data_test$exposure_start == -1 & data_test$reporting.date == repDate, "exposure_start"] <- categorias[categorias[,1] == repDate, 2]
}

data_test$age <- as.numeric(data_test$age)

# removendo todos os valores -1, trocando pela mediana:
colunas <- 1:15; 
for (i in colunas[-3]){
  mediana <- median(data_test[data_test[,i] != -1, i])
  data_test[data_test[,i] == -1, i] <- mediana
}

dataTest_opt <- data_test

       X        case_in_country  reporting.date          country   
 Min.   :   7   Min.   : 1.000   Min.   : 4.000   Japan      : 47  
 1st Qu.: 407   1st Qu.: 1.000   1st Qu.: 7.000   China      : 43  
 Median : 838   Median : 3.000   Median :11.000   Germany    : 30  
 Mean   :1458   Mean   : 3.268   Mean   : 9.673   USA        : 27  
 3rd Qu.:2935   3rd Qu.: 4.000   3rd Qu.:12.000   Singapore  : 26  
 Max.   :3392   Max.   :10.000   Max.   :14.000   South Korea: 25  
                NA's   :43                        (Other)    :117  
    gender         age        symptom_onset    If_onset_approximated
 female:115   Min.   :1.000   Min.   : 0.000   Min.   :0.00000      
 male  :136   1st Qu.:4.000   1st Qu.: 5.000   1st Qu.:0.00000      
 NA's  : 64   Median :5.000   Median : 7.000   Median :0.00000      
              Mean   :5.282   Mean   : 7.059   Mean   :0.03788      
              3rd Qu.:7.000   3rd Qu.:10.000   3rd Qu.:0.00000      
              Max.   :8.000   Max.   :13.0

[1] 315  17

# 3. Balancing Classes

Inspecting train data:

In [29]:
table(dataTrain_opt$label)


       dead onTreatment   recovered 
         43         855         109 

We'll use **Oversampling**. Each low-balanced class ('dead' and 'recovered') are replicated with different replication values:

In [32]:
underData1 <- dataTrain_opt[dataTrain_opt$label == "dead",] # "dead"
underData2 <- dataTrain_opt[dataTrain_opt$label == "recovered",] # "recovered"
repl1 <- 11 # replicated "dead" samples
repl2 <- 6 # replicated "recovered" samples

Balancing each class:

In [34]:
# 'dead' class:
selectedIndex1 <- sample(1:nrow(underData1), repl1*nrow(underData1), replace=TRUE)
oversampledData1 <- underData1[selectedIndex1,]

# 'recovered' class:
selectedIndex2 <- sample(1:nrow(underData2), repl2*nrow(underData2), replace=TRUE)
oversampledData2 <- underData2[selectedIndex2,]

Creating the training data set:

In [35]:
dataTrain_opt <- rbind(oversampledData1,oversampledData2, dataTrain_opt[dataTrain_opt$label == "onTreatment",])
table(dataTrain_opt$label)


       dead onTreatment   recovered 
        473         855         654 

# 4. Baseline Model: Decision Tree

Importing libraries:

In [38]:
set.seed(123)

library("caret")
library("reshape2")
library("ggplot2")
library("rpart")
library("rpart.plot")
library("randomForest")
library("ramify")
library("gridExtra")
library("randomForest")

Auxiliary function for predictions:

In [40]:
PredEval <- function(model, data, isDecisionTree = TRUE){
  # Prediction
  pred <- predict(model, data)
  
  if (isDecisionTree){
  # data classification
  valClasses <- argmax(pred)
  prediction <- NULL
  prediction[valClasses==1] <- "dead"
  prediction[valClasses==2] <- "onTreatment"
  prediction[valClasses==3] <- "recovered"
  }
  else {
    prediction <- pred
  }
  
  # Prediction results
  cm <- confusionMatrix(data = as.factor(prediction), 
                        reference = as.factor(data$label))
  
  out <- list(mean(cm$byClass[,"Sensitivity"]),  # normalized accuracy
              cm$overall["Accuracy"],            # accuracy
              cm$table,                          # confusion matrix
              cm$byClass[,1])                    # TPR (true positive rate)
  names(out) <- c("AccNorm", "Acc", "CM", "TPR")
  
  return(out)
}

Declaring the lists to store the results:

In [43]:
resultsTrain_opt <- list()
resultsVal_opt <- list()
resultsTest_opt <- list()

Training a simple decision tree model as baseline:

In [39]:
resultsTrain_opt <- list()
resultsVal_opt <- list()
resultsTest_opt <- list()

baseline_model_opt <- rpart(formula = label ~ case_in_country + reporting.date +
                          country + gender + age + symptom_onset +
                          If_onset_approximated + hosp_visit_date +
                          international_traveler + domestic_traveler +
                          exposure_start + exposure_end+traveler +
                          visiting.Wuhan + from.Wuhan, 
                        data = dataTrain_opt, method="class",
                        control = rpart.control(minsplit=2, cp=0.0, xval = 10),
                        parms= list(split="information"))

In [44]:
train_results_opt <- PredEval(baseline_model_opt, dataTrain_opt)
val_results_opt <- PredEval(baseline_model_opt, dataVal_opt)
test_results_opt <- PredEval(baseline_model_opt, dataTest_opt)

Storing results:

In [46]:
resultsTrain_opt[[1]] <- train_results_opt
resultsVal_opt[[1]] <- val_results_opt
resultsTest_opt[[1]] <- test_results_opt

Results for baseline model:

In [48]:
resultsTrain_opt[[1]]

$AccNorm
[1] 0.9944221

$Acc
 Accuracy 
0.9929364 

$CM
             Reference
Prediction    dead onTreatment recovered
  dead         473           5         0
  onTreatment    0         842         1
  recovered      0           8       653

$TPR
       Class: dead Class: onTreatment   Class: recovered 
         1.0000000          0.9847953          0.9984709 


In [50]:
resultsVal_opt[[1]]

$AccNorm
[1] 0.7367395

$Acc
 Accuracy 
0.9007937 

$CM
             Reference
Prediction    dead onTreatment recovered
  dead           7           5         1
  onTreatment    6         199         7
  recovered      0           6        21

$TPR
       Class: dead Class: onTreatment   Class: recovered 
         0.5384615          0.9476190          0.7241379 


In [52]:
resultsTest_opt[[1]]

$AccNorm
[1] 0.8015014

$Acc
 Accuracy 
0.9174603 

$CM
             Reference
Prediction    dead onTreatment recovered
  dead           8           3         0
  onTreatment    5         248         6
  recovered      0          12        33

$TPR
       Class: dead Class: onTreatment   Class: recovered 
         0.6153846          0.9429658          0.8461538 


Great results on Training set, still room for improvement on Validation and Test set. Also, normalized accuracy analysis is of upmost importance in this problem, since we have a strongly unbalanced data set.

# 5. Random Forest

Training the random forest model:

In [53]:
rf_model_opt <- randomForest(formula = label ~ case_in_country + reporting.date +
                           country + gender + age + symptom_onset +
                           If_onset_approximated + hosp_visit_date +
                           international_traveler + domestic_traveler +
                           exposure_start + exposure_end+traveler +
                           visiting.Wuhan + from.Wuhan, 
                         data = dataTrain_opt)

Predicting and storing metrics:

In [56]:
train_results_opt <- PredEval(rf_model_opt, dataTrain_opt, isDecisionTree = FALSE)
val_results_opt <- PredEval(rf_model_opt, dataVal_opt, isDecisionTree = FALSE)
test_results_opt <- PredEval(rf_model_opt, dataTest_opt, isDecisionTree = FALSE)

resultsTrain_opt[[2]] <- train_results_opt;
resultsVal_opt[[2]] <- val_results_opt;
resultsTest_opt[[2]] <- test_results_opt;

Results for random forest model:

In [57]:
resultsTrain_opt[[2]]

$AccNorm
[1] 0.9829068

$Acc
 Accuracy 
0.9788093 

$CM
             Reference
Prediction    dead onTreatment recovered
  dead         473          16         0
  onTreatment    0         819         6
  recovered      0          20       648

$TPR
       Class: dead Class: onTreatment   Class: recovered 
         1.0000000          0.9578947          0.9908257 


In [58]:
resultsVal_opt[[2]]

$AccNorm
[1] 0.8078355

$Acc
 Accuracy 
0.9087302 

$CM
             Reference
Prediction    dead onTreatment recovered
  dead           9           4         1
  onTreatment    4         197         5
  recovered      0           9        23

$TPR
       Class: dead Class: onTreatment   Class: recovered 
         0.6923077          0.9380952          0.7931034 


In [59]:
resultsTest_opt[[2]]

$AccNorm
[1] 0.817003

$Acc
 Accuracy 
0.8952381 

$CM
             Reference
Prediction    dead onTreatment recovered
  dead           9           6         0
  onTreatment    3         240         6
  recovered      1          17        33

$TPR
       Class: dead Class: onTreatment   Class: recovered 
         0.6923077          0.9125475          0.8461538 


Great improvement in Validation set, in terms of normalized accuracy, maily due to better predictions of 'dead' class.